In [9]:
import os

In [10]:
%pwd

'd:\\STUDY CODING\\projects\\ML\\End-To-End-Machine-Learning-with-MLflow-\\research'

In [11]:
os.chdir("../")

In [12]:
%pwd

'd:\\STUDY CODING\\projects\\ML\\End-To-End-Machine-Learning-with-MLflow-'

In [1]:
import dagshub
dagshub.init(repo_owner='Ashish570raj', repo_name='End-To-End-Machine-Learning-with-MLflow-', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\STUDY CODING\projects\ML\End-To-End-Machine-Learning-with-MLflow-\myenv\Lib\site-packages\rich\live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=6b315121-bb69-4800-8fd2-413b60aeb631&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1dcd145940cd48d2f5090333b4cb0cfc58d419beab09d80337109f9394862a0b




Accessing as Ashish570raj

Initialized MLflow to track repo "Ashish570raj/End-To-End-Machine-Learning-with-MLflow-"

Repository Ashish570raj/End-To-End-Machine-Learning-with-MLflow- initialized!

🏃 View run defiant-moose-251 at: https://dagshub.com/Ashish570raj/End-To-End-Machine-Learning-with-MLflow-.mlflow/#/experiments/0/runs/9e11faf47f504030a0a3183445134938
🧪 View experiment at: https://dagshub.com/Ashish570raj/End-To-End-Machine-Learning-with-MLflow-.mlflow/#/experiments/0


In [ ]:
# os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Ashish570raj/End-To-End-Machine-Learning-with-MLflow-.mlflow"
# os.environ["MLFLOW_TRACKING_USERNAME"]="Ashish570raj"
# os.environ["MLFLOW_TRACKING_PASSWORD"]="3230b6c77f1e8ef8993811ab66d84fc2012fb452"

In [2]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [3]:
from  mlproject.constants import * 
from mlproject.utils.common import read_yaml,create_directories,save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Ashish570raj/End-To-End-Machine-Learning-with-MLflow-.mlflow",
           
        )

        return model_evaluation_config


In [5]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [6]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-21 08:21:23,563: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-21 08:21:23,571: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-21 08:21:23,577: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-21 08:21:23,581: INFO: common: created directory at: artifacts]
[2025-05-21 08:21:23,585: INFO: common: created directory at: artifacts/model_evaluation]
[2025-05-21 08:21:24,529: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/05/21 08:21:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/05/21 08:21:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 5
Created version '5' of model 'ElasticnetModel'.


🏃 View run delightful-bass-577 at: https://dagshub.com/Ashish570raj/End-To-End-Machine-Learning-with-MLflow-.mlflow/#/experiments/0/runs/358956cb34a743e391a7c9f810b26ff7
🧪 View experiment at: https://dagshub.com/Ashish570raj/End-To-End-Machine-Learning-with-MLflow-.mlflow/#/experiments/0
